# SAR2SAR: a self-supervised despeckling algorithm for SAR images
## Emanuele Dalsasso, Loïc Denis, Florence Tupin

Please note that the training set is only composed of **GRD** SAR images, thus this testing code is specific to this data.

## Resources
- [Paper (ArXiv)](https://arxiv.org/abs/2006.15037)

To cite the article:

    @article{dalsasso2020sar2sar,
        title={{SAR2SAR}: a self-supervised despeckling algorithm for {SAR} images},
        author={Emanuele Dalsasso and Loïc Denis and Florence Tupin},
        journal={arXiv preprint arXiv:2006.15037},
        year={2020}
    }

## 1. Install required libraries

In [ ]:
!pip install -r requirements.txt

## 2. Set up the data enviornment
The model.py contains the run_model function. Here is the docstring for the run_model function:
    
    Runs the despeckling algorithm

    Arguments:
        input_dir: Path to a directory containing the files to be despeckled. Files need to be in .npy
                   format
        save_dir: Path to a directory where the files will be saved
        checkpoint_dir: Path to a directory containing the tensorflow checkpoints, if left as None, the
                        despeckling algorithm will use the grd_checkpoint directory
        stride: U-Net is scanned over the image with a default stride of 64 pixels when the image dimension
                exceeds 256. This parameter modifies the default stride in pixels. Lower pixel count = higher quality
                results, at the cost of higher runtime
        store_noisy: Whether to store the "noisy" or input in the save_dir. Default is True
        generate_png: Whether to generate PNG of the outputs in the save_dir. Default is True
        debug: Whether to generate print statements at runtime that communicate what is going on

    Returns:
        None

**You will need to set the input_dir and save_dir filepaths.** Here is an example of what that could look like.

In [ ]:
import os
from pathlib import Path

# current folder this file is in
current_dir = Path(os.getcwd())

# set the path of the input and save directories
example_input_dir = str(current_dir / "src" / "test_data" / "grd_test_data")
example_save_dir = str(current_dir / "example_output")

# set the path of your own input and save directory
my_input_dir = str()
my_save_dir = str()

print(f"Input directory set to: {my_input_dir}")
print(f"Save directory set to: {my_save_dir}")

## 3. Run the example model
Run the example model to make sure that everything has been installed correctly and is ready to run. This code was originally written for Tensorflow V1, so the tensorflow library will throw a lot of warnings.
When the model is done, you should see a folder named example_output with noisy_ and denoised_ files inside of it.

***The model will print this line when it has finished:***

[!!!] Done

In [ ]:
import tensorflow as tf
tf.compat.v1.reset_default_graph()
from src.model import run_model

run_model(example_input_dir, example_save_dir)

## 4. Run the model on your data

In [ ]:
tf.compat.v1.reset_default_graph()
run_model(my_input_dir, my_save_dir)